In [ ]:
import pandas as pd
data_dir = '/Users/adamgifford/Documents/DataScience/DataIncubator/Winter 2021/12_Day_Challenge/data/'

In [ ]:
files_dct = {}
files_dct['daily'] = 'daily_IBM.csv'
files_dct['daily_adj'] = 'daily_adjusted_IBM.csv'
files_dct['intraday'] = 'intraday_5min_IBM.csv'
files_dct['monthly'] = 'monthly_IBM.csv'

In [ ]:
def load_file(data_dir,filename):
    df = pd.read_csv(data_dir + filename)
    return df

In [ ]:
df = load_file(data_dir,files_dct['daily'])

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.head()

In [ ]:
import numpy as numpy
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE
from bokeh.plotting import figure
from bokeh.models.glyphs import Line
output_notebook()

In [ ]:
colors_dict = {}
colors_dict['open'] = 'firebrick'
colors_dict['close'] = 'navy'
colors_dict['adj_open'] = 'indianred'
colors_dict['adj_close'] = 'darkslateblue'

In [ ]:
import json
def get_key(json_file='alpha_vantage_config.json',json_key='alpha_vantage_info'):
    with open(json_file) as f:
        data = json.load(f)
    
    user_values = data[json_key]
    return user_values['api_key']

In [ ]:
json_file = '/Users/adamgifford/Documents/DataScience/DataIncubator/Winter 2021/12_Day_Challenge/alpha_vantage_config.json'
key = get_key(json_file=json_file)

In [ ]:
base_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'
adj_base_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED'

In [ ]:
# app.vars['symbol'] = request.form['symbol']
# # open           = option1
# # close          = option2
# # adjusted close = option3
# app.vars['open'] = request.form.get('option1')
# app.vars['close'] = request.form.get('option2')
# app.vars['ajd_close'] = request.form.get('option3')

appvars = {}
appvars['symbol'] = 'IBM'
appvars['close'] = 'close'
appvars['ajd_close'] = None
appvars['open'] = 'open'
appvars['adj_open'] = None

In [ ]:
import requests
# get daily adjusted price from alphavantage
ticker = 'GOOG'
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey={}'.format(ticker, key)
response = requests.get(url).json()

In [ ]:
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource, CrosshairTool
from bokeh.models.callbacks import CustomJS
import numpy as np

to_plot = ['open', 'close']
ymin = min(df[['open','close']].min())
ymax = max(df[['open','close']].max())

TOOLS = 'box_zoom,reset'
# create a new plot (with a title) using figure
p = figure(x_axis_type="datetime", title="Stock Open & Close", plot_height=350, plot_width=700, tools=TOOLS)

l1 = p.line(df.timestamp, df.open, line_width=2, line_color=colors_dict['open'], alpha = 0.5)
l2 = p.line(df.timestamp, df.close, line_width=2, line_color=colors_dict['close'], alpha = 0.5)

# # add a hover tool that sets the link data for a hovered circle
# code = """
# const links = %s
# const data = {'x0': [], 'y0': [], 'x1': [], 'y1': []}
# const indices = cb_data.index.indices
# for (var i = 0; i < indices.length; i++) {
#     const start = indices[i]
#     for (var j = 0; j < links[start].length; j++) {
#         const end = links[start][j]
#         data['x0'].push(circle.data.x[start])
#         data['y0'].push(circle.data.y[start])
#         data['x1'].push(circle.data.x[end])
#         data['y1'].push(circle.data.y[end])
#     }
# }
# segment.data = data
# """ % links

# callback = CustomJS(args={'circle': cr.data_source, 'segment': sr.data_source}, code=code)

js = '''
var geometry = cb_data['geometry'];
console.log(geometry);
var data = line_source.data;
var x = data['x'];
console.log(x);
if (isFinite(geometry.x)) {
  for (i = 0; i < x.length; i++) {
    x[i] = geometry.x;
  }
  line_source.change.emit();
}
'''

source = ColumnDataSource(dict(
        x0=[df['timestamp'].min()],
        x1=[df['timestamp'].min()],
        y0=[ymin],
        y1=[ymax],
        op=df[df['timestamp']==df['timestamp'].min()]['open'].values,
        cl=df[df['timestamp']==df['timestamp'].min()]['close'].values,
    )
)

vl = p.segment('x0', 'y0', 'x1', 'y1', line_color='black', line_width=2, source=source)

js = '''
var geometry = cb_data['geometry']
var data = source.data;
var new_data = {'x0': [], 'y0': [], 'x1': [], 'y1': [], 'op': [], 'cl': []}
new_data['x0'] = geometry['x'];
new_data['x1'] = geometry['x'];
new_data['y0'] = data['y0'];
new_data['y1'] = data['y1'];
new_data['op'] = data['op'];
new_data['cl'] = data['cl'];
source.data=new_data;
'''
callback = CustomJS(code=js, args={'source': source})

tips = [("(x,y)",'($x,$y)'),("date",'@x0{%F}'),("open",'@op'),("close",'@cl')]
# hover = HoverTool(tooltips=tips, mode='hline', point_policy='follow_mouse', renderers=[vl], formatters = {'@x0':'datetime'})
hover = HoverTool(tooltips=tips, mode='hline', point_policy='follow_mouse', renderers=[vl], formatters = {'@x0':'datetime'}, callback=callback)
p.add_tools(hover)

# source.js_on_change('data', callback)
# s1.selected.js_on_change('indices', callback)
show(p) # show the results

In [ ]:
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource, CrosshairTool
from bokeh.models.callbacks import CustomJS
import numpy as np

to_plot = ['open', 'close']

# source = ColumnDataSource(data=df)
source = ColumnDataSource({'xs': [df['timestamp'], df['timestamp']], 
                           'ys': [df['open'], df['close']], 
                           'colors': [colors_dict['open'], colors_dict['close']]
                          })

TOOLS = 'box_zoom,reset'
# create a new plot (with a title) using figure
p = figure(x_axis_type="datetime", title="Stock Open & Close", plot_height=350, plot_width=700, tools=TOOLS)

l = p.multi_line(xs='xs',ys='ys', line_color='colors', line_width=2, alpha=0.5, source=source)

tips = [("date",'@xs'),("(open, close)",'@ys')]
hover = HoverTool(tooltips=None, mode='vline', point_policy='snap_to_data', line_policy='nearest', renderers=[l])
p.add_tools(hover)

# source.js_on_change('data', callback)
# s1.selected.js_on_change('indices', callback)
show(p) # show the results

In [ ]:
import bokeh
bokeh.__version__

In [ ]:
df[df['timestamp']==df['timestamp'].min()]['open']

In [ ]:
df.head()

In [ ]:
source.data['y']

In [ ]:
df['open_color'] = 'firebrick'
df['open_hover_color'] = 'red'

df['close_color'] = 'navy'
df['close_hover_color'] = 'blue'

df['width'] = 2
df['alpha'] = 0.4
df['hover_alpha'] = 1
df.head()

source.column_names

In [ ]:
[5]*10

In [ ]:
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource
import numpy as np

source = ColumnDataSource(data=df)
ymin = min(df[['open','close']].min())
ymax = max(df[['open','close']].max())

df['x'] = df['timestamp'][0]
df['y'] = np.linspace(ymin,ymax,len(df))

# create a new plot (with a title) using figure
p = figure(x_axis_type="datetime", title="Stock Open & Close", plot_height=350, plot_width=700)

# add a line renderer
# l1 = p.line('timestamp', 'open', line_width=2, line_color="firebrick", hover_line_color="red",
#             hover_alpha = 1, alpha = 0.4, source = df)
# l2 = p.line('timestamp', 'close', line_width=2, line_color="navy", hover_line_color="blue",
#             hover_alpha = 1, alpha = 0.4, source = df)
# l3 = p.line([df.timestamp[0]]*100,np.linspace(ymin, ymax, 100), line_color = 'black',
#             line_width=2)

l1 = p.multi_line(xs=[df.timestamp, df.timestamp], ys=[df.open, df.close], 
                color=["firebrick", "navy"], line_alpha=[0.4, 0.4],
                line_width=[2, 2])
s2 = ColumnDataSource(data={x=[df['timestamp'][0]]*len(df), y= \


[0.5, 0.5]})
l2 = p.line([df.timestamp[0]]*100,np.linspace(ymin, ymax, 100), line_color = 'black',
            line_width=2)

js = '''
var geometry = cb_data['geometry'];
console.log(geometry);
var data = line_source.data;
var x = data['x'];
console.log(x);
if (isFinite(geometry.x)) {
  for (i = 0; i < x.length; i++) {
    x[i] = geometry.x;
  }
  line_source.change.emit();
}
'''

# hover = HoverTool(tooltips=None, 
#                       point_policy='follow_mouse', 
#                       callback=CustomJS(code=js, args={'line_source': line_source}))

# l = p.multi_line(xs=[df.timestamp, df.timestamp], ys=[df.open, df.close], 
#                 color=["firebrick", "navy"], line_alpha=[0.4, 0.4],
#                 line_width=[2, 2])

# p.add_tools(HoverTool(tooltips=[("date",'@xs{%F}'),("open, close",'@ys')],             renderers=[l], mode='vline', formatters = {'@timestamp':'datetime'}))
# p.add_tools(HoverTool(tooltips=[("date",'@timestamp{%F}'),("open",'@open'),("close",'@close')], renderers=[l1], mode='vline', formatters = {'@timestamp':'datetime'}))
# p.add_tools(HoverTool(tooltips=[("close","@close")], renderers=[l2], mode='vline', formatters = {'@timestamp':'datetime'}, anchor='bottom_right'))

show(p) # show the results

In [ ]:
l

In [ ]:
h.show+

In [ ]:
l.data_source.data['xs'][0]

In [ ]:
foo

In [ ]:
from bokeh.io import show, output_file, save, show
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models.callbacks import CustomJS

# output_file("cross_hair.html")

x = list(range(12))
y = [v**2 for v in x]

NUM_PLOTS = 3

# Define a DataSource
data = dict(x=[0]*NUM_PLOTS)

line_source = ColumnDataSource(data=data)

js = '''
var geometry = cb_data['geometry'];
console.log(geometry);
var data = line_source.data;
var x = data['x'];
console.log(x);
if (isFinite(geometry.x)) {
  for (i = 0; i < x.length; i++) {
    x[i] = geometry.x;
  }
  line_source.change.emit();
}
'''


plots = []
for i in range(NUM_PLOTS):
    plot = figure(plot_width=250, plot_height=250, title=None)
    plot.segment(x0='x', y0=0, x1='x', y1=200, color='red', line_width=1, source=line_source)
    plot.circle(x, y, size=10, color="navy", alpha=0.5)
    hover = HoverTool(tooltips=None, 
                      point_policy='follow_mouse', 
                      callback=CustomJS(code=js, args={'line_source': line_source}))
    plot.add_tools(hover)
    plots.append(plot)

show(column(*plots))

In [ ]:
from bokeh.models import ColumnDataSource, CustomJS, HoverTool
from bokeh.plotting import figure, show

# define some points and a little graph between them
x = [2, 3, 5, 6, 8, 7]
y = [6, 4, 3, 8, 7, 5]
links = {
    0: [1, 2],
    1: [0, 3, 4],
    2: [0, 5],
    3: [1, 4],
    4: [1, 3],
    5: [2, 3, 4]
}

p = figure(plot_width=400, plot_height=400, tools="", toolbar_location=None, title='Hover over points')

source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1': []})
sr = p.segment(x0='x0', y0='y0', x1='x1', y1='y1', color='olive', alpha=0.6, line_width=3, source=source, )
cr = p.circle(x, y, color='olive', size=30, alpha=0.4, hover_color='olive', hover_alpha=1.0)

# add a hover tool that sets the link data for a hovered circle
code = """
const links = %s
const data = {'x0': [], 'y0': [], 'x1': [], 'y1': []}
const indices = cb_data.index.indices
for (var i = 0; i < indices.length; i++) {
    const start = indices[i]
    for (var j = 0; j < links[start].length; j++) {
        const end = links[start][j]
        data['x0'].push(circle.data.x[start])
        data['y0'].push(circle.data.y[start])
        data['x1'].push(circle.data.x[end])
        data['y1'].push(circle.data.y[end])
    }
}
segment.data = data
""" % links

callback = CustomJS(args={'circle': cr.data_source, 'segment': sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None, callback=callback, renderers=[cr]))

show(p)

In [ ]:
import numpy as np
import pandas as pd
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from datetime import datetime

df2 = pd.DataFrame(data = np.random.rand(10, 3),
columns = ('Argentina', 'Turkey' , 'Mexico'),
index = pd.DatetimeIndex(df['timestamp'][:10]))

source = ColumnDataSource({'date': [df2.index.values] * len(df2.columns),
'countries': ['Argentina', 'Turkey' , 'Mexico'],
'data': [df2[name].values for name in df2],
'colors': ['red', 'green', 'blue']})

p = figure(plot_width = 1000, x_axis_type = 'datetime', tools = 'hover')
multilines = p.multi_line(xs = 'date', ys = 'data', color = 'colors', source = source)
circles1 = p.circle(df2.index.values, df2['Argentina'].values, color = 'red', size = 8)
circles2 = p.circle(df2.index.values, df2['Turkey'].values, color = 'green', size = 8)
circles3 = p.circle(df2.index.values, df2['Mexico'].values, color = 'blue', size = 8)

hover = p.select(dict(type = HoverTool))
hover.renderers = [multilines]
hover.tooltips = [('Date', '@x{%F}'), ('Value', '@y')]
hover.formatters = {'x': 'datetime', 'y': 'printf'}

show(p)

In [ ]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
        
source = ColumnDataSource(
                data=dict(
                    x=[[1, 3, 2], [3, 4, 6, 6]],
                    y=[[2, 1, 4], [4, 7, 8, 5]],
                    desc=['a', 'b'],
                )
            )
        
hover = HoverTool(
                tooltips=[
                    ("index", "$index"),
                    ("(x,y)", "($x, $y)"),
                    ("desc", "@desc"),
                ],
            mode='hline')
        
p = figure(plot_width=400, plot_height=400, tools=[hover],
                   title="Mouse over the dots")
        
p.multi_line('x', 'y', line_width=2, source=source)
       
show(p)

In [ ]:
import numpy as np
from bokeh.io import show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.plotting import figure
from bokeh.layouts import column

source1 = ColumnDataSource(data={"freq":np.random.uniform(2, 10, 10), "amp":np.random.uniform(-2, 2, 10)})
source2 = ColumnDataSource(data={"x":np.linspace(-5, 5, 200), "y":np.zeros(200)})

fig1 = figure(plot_height=200)
fig2 = figure(plot_height=200)
circle = fig1.circle("freq", "amp", source=source1, size=10)
line = fig2.line("x", "y", source=source2)

def callback(source1=source1, source2=source2):
    indices = cb_data.index["1d"].indices
    if len(indices) > 0:
        i = indices[0]
        f = source1.data.freq[i]
        a = source1.data.amp[i]
        for j in range(len(source2.data.x)):
            source2.data.y[j] = a * Math.sin(f * source2.data.x[j])
        source2.change.emit()

hover = HoverTool(callback=CustomJS.from_py_func(callback))
fig1.add_tools(hover)

show(column(fig1, fig2))